In [438]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import random

class MeanAggregator(nn.Module):
    def __init__(self, features, cuda=False, gcn=False):
        super(MeanAggregator, self).__init__()
        self.features = features
        self.cuda = cuda
        self.gcn = gcn

    def forward(self, nodes, to_neighs, num_sample=10):
        _set = set
        if not num_sample is None:
            _sample = random.sample
            samp_neighs = [_set(_sample(to_neigh, num_sample)) if len(to_neigh) >= num_sample else to_neigh for to_neigh in to_neighs]
            # 注意：此处先执行前面的判断语句，后执行for to_neigh in to_neighs形成list
        else:
            samp_neighs = to_neighs
        if self.gcn:
            samp_neighs = [set.union(samp_neigh, _set([nodes[i]])) for i, samp_neigh in enumerate(samp_neighs)]
        unique_nodes_list = list(set.union(*samp_neighs))
        unique_nodes = {n:i for i, n in enumerate(unique_nodes_list)}
        mask = Variable(torch.zeros(len(samp_neighs), len(unique_nodes)))
        column_indices = [unique_nodes[n] for samp_neigh in samp_neighs for n in samp_neigh]
        row_indices = [i for i in range(len(samp_neighs)) for j in range(len(samp_neighs[i]))]
        mask[row_indices, column_indices] = 1
        if self.cuda:
            mask = mask.cuda()
        num_neigh = mask.sum(1, keepdims=True)
        mask = mask.div(num_neigh)
        if self.cuda:
            embed_matrix = self.features(torch.LongTensor(unique_nodes_list).cuda())
        else:
            embed_matrix = self.features(torch.LongTensor(unique_nodes_list))
        to_feats = mask.mm(embed_matrix)
        return to_feats

In [439]:
# 理解关于set的随机采样
a = set
b = random.sample
c = [1,2,3,4,5]
d = [a(b(c,3))]
print(d)
# print([a(c)+a(c)])

[{2, 3, 4}]


In [440]:
# 理解set的并集操作
e = [2,3,4,5,6]
f = [a(c), a(e)]
print(f)
g = list(set.union(*f))
print(g)
h = set.union(a(c), a([7]))
print(h)

[{1, 2, 3, 4, 5}, {2, 3, 4, 5, 6}]
[1, 2, 3, 4, 5, 6]
{1, 2, 3, 4, 5, 7}


In [441]:
import torch
import torch.nn as nn
from torch.nn import init
import torch.nn.functional as F

class Encoder(nn.Module):
    def __init__(self, features, feature_dim, embed_dim, adj_lists, aggregator, num_sample=10, base_model=None, gcn=False, cuda=False):
        super(Encoder, self).__init__()
        self.features = features
        self.feat_dim = feature_dim
        self.adj_lists = adj_lists
        self.aggregator = aggregator
        self.num_sample = num_sample
        if base_model != None:
            self.base_model = base_model
        self.gcn = gcn
        self.embed_dim = embed_dim
        self.cuda = cuda
        self.aggregator.cuda = cuda
        self.weight = nn.Parameter(torch.FloatTensor(embed_dim, self.feat_dim if self.gcn else 2*self.feat_dim))
        # 注意：此处将","后面的语句作为整体执行
        init.xavier_uniform_(self.weight) # 初始化操作

    def forward(self, nodes):
        # neigh_feats = self.aggregator.forward(nodes, self.adj_lists, self.num_sample)
        neigh_feats = self.aggregator.forward(nodes, [self.adj_lists[int(node)] for node in nodes], self.num_sample)
        if not self.gcn:
            if self.cuda:
                self_feats = self.features(torch.LongTensor(nodes).cuda())
            else:
                self_feats = self.features(torch.LongTensor(nodes))
                # print(self_feats.shape) # (34, 16)
            combined = torch.cat([self_feats, neigh_feats], dim=1)
            # print(combined.shape) # (34, 32)
        else:
            combined = neigh_feats
        # print(combined.shape) # (34, 32)
        # print(self.weight.shape) # (16, 32)
        combined = F.relu(self.weight.mm(combined.t()))
        # print(combined.shape) # (16, 34)
        return combined

In [442]:
import numpy as np
import time
import random
from sklearn.metrics import f1_score
from collections import defaultdict
import networkx as nx
from torch.nn import Linear

class SupervisedGraphSage(nn.Module):
    def __init__(self, num_classes, enc):
        super(SupervisedGraphSage, self).__init__()
        self.enc = enc
        self.xent = nn.CrossEntropyLoss()
        self.weight = nn.Parameter(torch.FloatTensor(num_classes, enc.embed_dim))
        init.xavier_uniform_(self.weight)

    def forward(self, nodes):
        embeds = self.enc(nodes)
        # print(embeds.shape) # (16, 34)
        # print(self.weight.shape) # (4, 16)
        scores = self.weight.mm(embeds)
        # print(scores.shape) # (4, 34)
        # return scores.t() # (34, 4)
        scores_softmax = torch.exp(scores.t())/torch.sum(torch.exp(scores.t()), dim=1).reshape(-1, 1)
        # print(scores_softmax.shape)
        # print(scores_softmax.sum(dim=1))
        return scores_softmax # (34, 4)

    def loss(self, nodes, labels):
        scores = self.forward(nodes)
        # print(scores.sum(dim=1))
        return self.xent(scores, labels)

In [443]:
def get_to_neighs(G):
    num_nodes = len(G.nodes)
    to_neighs = []
    for i in range(num_nodes):
        mid_set = set()
        for j in G.neighbors(i):
            mid_set.add(j)
        to_neighs.append(mid_set)
    return to_neighs

In [444]:
# def get_batch_nodes(graph):
#     batch_nodes = []
#     num_nodes = len(graph.nodes)
#     pos = nx.spring_layout(graph)
#     for i in range(num_nodes):
#         batch_nodes.append(list(pos[i]))
#     return batch_nodes

G = nx.karate_club_graph()
# batch_nodes = get_batch_nodes(G)
# print(batch_nodes)

from torch_geometric.datasets import KarateClub
dataset = KarateClub()
data = dataset[0]
labels = data.y
print(labels)

tensor([1, 1, 1, 1, 3, 3, 3, 1, 0, 1, 3, 1, 1, 1, 0, 0, 3, 1, 0, 1, 0, 1, 0, 0,
        2, 2, 0, 0, 2, 0, 0, 2, 0, 0])


In [445]:
def create_elements(graph = nx.karate_club_graph(), num_embed = 34, feature_dim = 16, embed_dim = 16, num_classes = 4, nodes=None):
    if nodes == None:
        nodes = []
        for i in range(len(graph.nodes)):
            nodes.append(i)
    split = int(len(nodes)/3*2)
    features = nn.Embedding(num_embed, embed_dim)
    to_neighs = get_to_neighs(graph)
    aggregator = MeanAggregator(features, nodes[:split], to_neighs)
    enc = Encoder(features=features, feature_dim=feature_dim, embed_dim=embed_dim, adj_lists=to_neighs, aggregator=aggregator)
    model = SupervisedGraphSage(num_classes=num_classes, enc=enc)
    # criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    return nodes, to_neighs, model, optimizer, features, split

def train(nodes, labels, model, optimizer):
    times = []
    losses = []
    for i in range(100):
        start_time = time.time()
        optimizer.zero_grad()
        loss = model.loss(nodes, labels)
        losses.append(loss)
        loss.backward()
        optimizer.step()
        end_time = time.time()
        times.append(end_time-start_time)
    # print(times)
    # print(losses)

def predict(nodes, labels, model):
    after_model = model(nodes)
    predicted = torch.argmax(after_model, dim=1)
    acc = (torch.sum(predicted == labels) / len(labels)).item()
    print(acc)

数据集：nx.karate_club_graph()

In [446]:
nodes, to_neights, model, optimizer, features,split = create_elements(graph = nx.karate_club_graph(), num_embed = 34, feature_dim = 16, embed_dim = 16, num_classes = 4, nodes=None)
train(nodes=nodes[:split], labels=labels[:split], model=model, optimizer=optimizer)
predict(nodes=nodes[split:], labels=labels[split:], model=model)

0.4166666567325592


数据集：nx.read_edgelist("./facebook/414.edges")

In [447]:
G_fb = nx.read_edgelist("./facebook/414.edges")
n = G_fb.number_of_nodes()
m = G_fb.number_of_edges()
print(n, m)
mapping = dict(zip(G_fb.nodes(), range(n)))
# print(mapping)
nx.relabel_nodes(G_fb, mapping, copy=False)
# print(G_fb.nodes)

nodes_fb, to_neights_fb, model_fb, optimizer_fb, features_fb, split_fb = create_elements(graph = G_fb, num_embed = 150, feature_dim = 16, embed_dim = 16, num_classes = 11, nodes=list(G_fb.nodes))
labels_fb = torch.tensor([random.randint(0, 10) for i in range(150)])
train(nodes=nodes_fb[:split_fb], labels=labels_fb[:split_fb], model=model_fb, optimizer=optimizer_fb)
predict(nodes=nodes_fb[:split_fb], labels=labels_fb[:split_fb], model=model_fb)

150 1693
0.9800000190734863


数据集：cora

In [462]:
def load_cora():
    num_nodes = 2708
    num_feats = 1433
    feat_data = np.zeros((num_nodes, num_feats)) # 此处不能用torch代替np
    labels = np.empty((num_nodes, 1), dtype=np.int64) # 此处不能用torch代替np
    node_map = {}
    label_map = {}
    with open("./cora/cora.content") as fp:
        for i, line in enumerate(fp):
            info = line.strip().split()
            info_label = info[-1]
            info = info[:-1]
            info = [int(x) for x in info]
            feat_data[i, :] = info[1:]
            # print(len(feat_data[i, :]))
            node_map[info[0]] = i
            if not info_label in label_map:
                label_map[info_label] = len(label_map) # len({}) = 1
            labels[i] = label_map[info_label]
    adj_lists = defaultdict(set) # 注意：此处不能用普通的set
    # print(adj_lists)
    # print(node_map)
    # print(label_map)
    # print(labels.shape)
    with open("./cora/cora.cites") as fp:
        for i, line in enumerate(fp):
            info = line.strip().split()
            info = [int(x) for x in info]
            paper1 = node_map[info[0]]
            paper2 = node_map[info[1]]
            adj_lists[paper1].add(paper2)
            adj_lists[paper2].add(paper1)
    return feat_data, labels, adj_lists

# def run_cora():
np.random.seed(1) # 应对用np.random生成的随机数
random.seed(1) # 应对用random生成的随机数
num_nodes = 2708
num_feats = 1433
feat_data, labels, adj_lists = load_cora()
features = nn.Embedding(num_nodes, num_feats) # torch.LongTensor等价于torch.tensor
features.weight = nn.Parameter(torch.FloatTensor(feat_data), requires_grad=False) # torch.FloatTensor等价于torch.Tensor
# 上一行的作用：加快训练速度，提升训练效果(F1)
# print(features)
# print(features.weight.sum(dim=1))

agg1 = MeanAggregator(features)
enc1 = Encoder(features=features, feature_dim=1433, embed_dim=128, adj_lists=adj_lists, aggregator=agg1) # gcn=False
print(enc1(nodes).t().shape)
agg2 = MeanAggregator(features=lambda nodes : enc1(nodes).t())
enc2 = Encoder(features=lambda nodes : enc1(nodes).t(), feature_dim=enc1.embed_dim, embed_dim=128, adj_lists=adj_lists, aggregator=agg2, base_model=enc1) # gcn=False

graphsage = SupervisedGraphSage(7, enc2)
rand_indices = np.random.permutation(num_nodes)
# print(num_nodes)
# print(rand_indices)
test = rand_indices[:1000]
val = rand_indices[1000:1500]
train = list(rand_indices[1500:])
optimizer = torch.optim.SGD(filter(lambda p : p.requires_grad, graphsage.parameters()), lr=0.7)
# parameters = filter(lambda p : p.requires_grad, graphsage.parameters())
# print(parameters)
# print(graphsage.parameters())
times = []
for batch in range(100):
    batch_nodes = train[:256]
    random.shuffle(train)
    start_time = time.time()
    optimizer.zero_grad()
    loss = graphsage.loss(batch_nodes, torch.LongTensor(labels[np.array(batch_nodes)].reshape(1, -1)[0]))# Variable
    loss.backward()
    optimizer.step()
    end_time = time.time()
    times.append(end_time-start_time)
    print(batch, loss.item())

# print(labels[val])
# print(labels[train]) # 二者在格式上没有区别
val_output = graphsage.forward(val)
# print(val_output.data.numpy().argmax(axis=1))
# print("-"*50)
# print(val_output.argmax(axis=1))
print("Validation F1:", f1_score(labels[val], val_output.argmax(axis=1), average="micro"))
print("Average batch time:", np.mean(times))

torch.Size([34, 128])
0 1.9485934972763062
1 1.947161316871643
2 1.944930911064148
3 1.9428558349609375
4 1.9426424503326416
5 1.9395396709442139
6 1.9390537738800049
7 1.9382331371307373
8 1.9358720779418945
9 1.9325929880142212
10 1.933243989944458
11 1.930747389793396
12 1.9301179647445679
13 1.924185872077942
14 1.921176791191101
15 1.9207282066345215
16 1.9116379022598267
17 1.9020146131515503
18 1.8978430032730103
19 1.8932890892028809
20 1.875340223312378
21 1.8655987977981567
22 1.8482781648635864
23 1.8441038131713867
24 1.836130142211914
25 1.8032073974609375
26 1.806523323059082
27 1.7950505018234253
28 1.7865225076675415
29 1.7405025959014893
30 1.7768893241882324
31 1.8011536598205566
32 1.7608423233032227
33 1.7367984056472778
34 1.7544971704483032
35 1.7346073389053345
36 1.7169190645217896
37 1.7079381942749023
38 1.732297658920288
39 1.683693528175354
40 1.6918920278549194
41 1.672461748123169
42 1.6427375078201294
43 1.698271632194519
44 1.6535955667495728
45 1.660947

下面均为帮助理解整个网络架构的测试代码

In [ ]:
to_neighs = get_to_neighs(G)
samp_neighs = [set(random.sample(to_neigh, 3)) if len(to_neigh) >= 3 else to_neigh for to_neigh in to_neighs]
# print(samp_neighs)
samp_neighs = [set.union(samp_neigh, set([nodes[i]])) for i, samp_neigh in enumerate(samp_neighs)]
# print(samp_neighs)

In [ ]:
unique_nodes_list = list(set.union(*samp_neighs))
# print(unique_nodes_list)
unique_nodes = {n:i for i, n in enumerate(unique_nodes_list)}
# print(unique_nodes)
mask = Variable(torch.zeros(len(samp_neighs), len(unique_nodes)))
column_indices = [unique_nodes[n] for samp_neigh in samp_neighs for n in samp_neigh]
row_indices = [i for i in range(len(samp_neighs)) for j in range(len(samp_neighs[i]))]
mask[row_indices, column_indices] = 1
num_neigh = mask.sum(1, keepdims=True)
mask = mask.div(num_neigh) # 对每个节点的影响权权重(此处为mean)，节点自身与采样节点的权重一致

In [ ]:
embed_matrix = features(torch.LongTensor(unique_nodes_list)) # 34个节点，每个节点都有唯一的编码
# print(embed_matrix)
to_feats = mask.mm(embed_matrix)
# print(mask.shape)
# print(embed_matrix.shape)
# print(to_feats.shape)

In [ ]:
weight = nn.Parameter(torch.FloatTensor(3, 2 if False else 4))
init.xavier_uniform_(weight)
# print(weight)

In [ ]:
self_feats = features(torch.LongTensor(nodes))
# print(self_feats.shape)
# print(features)
# print(torch.LongTensor(nodes))

In [ ]:
a = "asd qwert zxcv"
a = a.strip().split()
print(map(float, a[1:-1]))
c = np.zeros((3,3))
c[1,:]=map(float, a[1:-1])